In [1]:
import json
import keras

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

import numpy as np
from GNG_basic_model_definition import gng_basic
import shap
import matplotlib.pyplot as plt

import os


2024-04-10 11:14:18.125192: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 11:14:18.125253: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 11:14:18.125298: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 11:14:18.139273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gng_classifier = keras.models.load_model("trained_gng_basic/partial_models/gng_basic_partiallyTrained_epoch_001.hdf5")

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2024-04-10 11:14:29.794394: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 898 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2024-04-10 11:14:29.796777: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10391 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [3]:
img_path = "/home/tanfoni/homeRepo/tanfoni/Dataset_mtcnn_split/Test/Fake/mtcnn_stylegan3_folder_8_5752_face0.png"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

In [5]:
masker=shap.maskers.Image("inpaint_telea", img_array[0].shape)
explainer = shap.Explainer(gng_classifier,masker, output_names=["Real", "Fake"])
# shap_values = explainer(img_array, max_evals=1000, outputs=shap.Explanation.argsort.flip)


In [10]:
def shap_plot(model, explainer, X_shap, Y_shap, X_shap_print, shap_output_path, num_images_shap=1,j=0):
    slice = X_shap[j:j+num_images_shap]
    slice_print=X_shap_print[j:j+num_images_shap]
    y_slice = Y_shap[j:j+num_images_shap]
    shap_values = explainer(slice,
                            outputs=shap.Explanation.argsort.flip[:num_images_shap],
                            max_evals=1000)

    predictions = model.predict(slice)
    predictions_labels = np.where(predictions > 0.5, 1, 0)

    predictions_to_plot = []
    for i in range(num_images_shap):
        predictions_to_plot.append(
            "Pred:{} | True:{}".format(str(predictions_labels.transpose().tolist()[i]),
                                       str(y_slice.transpose().tolist()[i])))
    predictions_to_plot = np.array([[i] for i in predictions_to_plot])
    plt.figure()
    shap.image_plot(shap_values, pixel_values=slice_print, labels=predictions_to_plot, show=False)
    if not os.path.isdir(shap_output_path):
        os.makedirs(shap_output_path)
    savepath = os.path.join(shap_output_path, "Shap_results_{}_set{}.png".format("pipo",j))
    plt.savefig(savepath)
    plt.close("all")

    

In [11]:
norm_layer = keras.layers.Rescaling(scale=1/255)
to_print=norm_layer(img_array).numpy()
shap_plot(gng_classifier, explainer, img_array, np.array([[1]]), to_print, "shap_output", num_images_shap=1,j=0)

  0%|          | 0/998 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:59, 59.26s/it]               

1/1 [==============================] - 0s 45ms/step


In [8]:
def plot_shap_values(model, explainer, img_array, label, output_path="shap_plot.png"):
    # Ensure img_array is four-dimensional [batch_size, height, width, channels]
    if len(img_array.shape) == 3:
        img_array = np.expand_dims(img_array, axis=0)
    
    # Generate SHAP explanations for the image
    # This will directly call the explainer on the img_array
    shap_explanations = explainer(img_array)
    
    # Make a prediction using the model
    prediction = model.predict(img_array)
    
    # Determine the predicted class
    predicted_class = np.argmax(prediction, axis=1)[0]
    
    # Plot SHAP values
    # Note: shap.image_plot expects the raw explanation object
    shap.image_plot(shap_explanations, show=False)
    
    # Add title with prediction and true label
    plt.title(f"Predicted: {predicted_class}, True: {label}")
    
    # Save or show the plot
    plt.savefig(output_path)
    plt.close()

In [9]:
img_path = "/home/tanfoni/homeRepo/tanfoni/Dataset_mtcnn_split/Test/Fake/mtcnn_stylegan3_folder_8_5752_face0.png"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

plot_shap_values(gng_classifier, explainer, img_array, label="Fake")

2024-04-10 11:16:27.804222: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-04-10 11:16:29.099775: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-10 11:16:29.107418: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-04-10 11:16:29.110296: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-04-10 11:16:29.110363: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at /usr/local/cuda/bin/ptxas
2024-04-10 11:16:29.110658: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: /usr/local/cuda/bin/ptxas ptxas too ol

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:35, 35.50s/it]               


1/1 [==============================] - 2s 2s/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
